In [6]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
data_dir = '/opt/ml/movie-recommendation/data/train/'

# 1. Rating df 생성
rating_data = data_dir + "/train_ratings.csv"
raw_rating_df = pd.read_csv(rating_data)

raw_rating_df['rating'] = 1.0 # implicit feedback
raw_rating_df.drop(['time'],axis=1,inplace=True)

users = set(raw_rating_df.loc[:, 'user'])
items = set(raw_rating_df.loc[:, 'item'])

In [4]:
# Negative instance 생성
num_negative = 50
user_group_dfs = list(raw_rating_df.groupby('user')['item'])
first_row = True
user_neg_dfs = pd.DataFrame()

for u, u_items in tqdm(user_group_dfs):
    u_items = set(u_items)
    i_user_neg_item = np.random.choice(list(items - u_items), num_negative, replace=False)
    
    i_user_neg_df = pd.DataFrame({'user': [u]*num_negative, 'item': i_user_neg_item, 'rating': [0]*num_negative})
    if first_row == True:
        user_neg_dfs = i_user_neg_df
        first_row = False
    else:
        user_neg_dfs = pd.concat([user_neg_dfs, i_user_neg_df], axis = 0, sort=False)

sampled_rating_df = pd.concat([raw_rating_df, user_neg_dfs], axis = 0, sort=False)
sampled_rating_df


100%|██████████| 31360/31360 [05:26<00:00, 95.93it/s] 


In [7]:
sampled_rating_df.to_csv(os.path.join(data_dir, 'Negative Sampled Ratings.csv'), mode='w', index=False)